In [306]:
import datetime
from bs4 import BeautifulSoup as bs

today = datetime.date.today()
year = str(today.year)
month = str(today.month)
day = str(today.day)

if len(month) == 1:
    month = '0' + month
if len(day) == 1:
    day = '0' + day

In [308]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import io

pts_prevtable = pd.DataFrame()
pts_currtable = pd.DataFrame()

# Initialize the WebDriver
driver = webdriver.Chrome()

# Points Per Player
URL = f"https://www.nba.com/stats/players/traditional?DateFrom={month}%2F{day}%2F{str(int(year) - 1)}&DateTo={month}%2F{day}%2F{year}&PerMode=PerMinute"
driver.get(URL)

#make the range number so that it just pulls whatever is on the nba site
for i in range(9):
    
    # Wait for the tables to load (you may need to adjust the timeout)
    wait = WebDriverWait(driver, 10)
    
    # Get the page source
    page_source = driver.page_source

    soup = bs(page_source, 'html.parser')

    table = soup.find('table', {'class': 'Crom_table__p1iZz'})

    pts_currtable = pd.read_html(str(table))[0]
 
    pts_prevtable = pd.concat([pts_prevtable, pts_currtable], ignore_index=True)
    
    # Find the "Next Page" button and click it
    next_button = driver.find_element(By.XPATH, "//*[@id='__next']/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[5]/button[2]")
    next_button.click()


# Total PPG
URL = "https://www.nba.com/stats/teams/traditional?dir=A&sort=PTS"
driver.get(URL)

# Wait for the tables to load (you may need to adjust the timeout)
wait = WebDriverWait(driver, 10)

# Get the page source
page_source = driver.page_source

ppg_table = pd.read_html(page_source)


#Defense Statistics
URL = "https://www.nba.com/stats/teams/defense"
driver.get(URL)
    
# Wait for the tables to load (you may need to adjust the timeout)
wait = WebDriverWait(driver, 10)

# Get the page source
page_source = driver.page_source

def_table = pd.read_html(page_source)


driver.quit()


/var/folders/2l/tqrw4qgn61q4jrsxc447rydm0000gn/T/ipykernel_69609/3637622191.py:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pts_currtable = pd.read_html(str(table))[0]
/var/folders/2l/tqrw4qgn61q4jrsxc447rydm0000gn/T/ipykernel_69609/3637622191.py:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pts_currtable = pd.read_html(str(table))[0]
/var/folders/2l/tqrw4qgn61q4jrsxc447rydm0000gn/T/ipykernel_69609/3637622191.py:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pts_currtable = pd.read_html(str(table))[0]
/var/folders/2l/tqrw4qgn61q4jrsxc447rydm0000gn/T/ipykernel_69609/3637622191.py:30: FutureWarning: Passing 

In [321]:
import json

URL = "https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=PerGame&Scope=S&Season=2023-24&SeasonType=Regular%20Season&StatCategory=PTS"
json = requests.get(URL)

parsed_json = json.json()

In [330]:
data = parsed_json['resultSet']['rowSet']
headers = parsed_json['resultSet']['headers']

df = pd.DataFrame(data, columns=headers)

df['PTS/MIN'] = df['PTS']/df['MIN']
df = df[['PLAYER', 'TEAM', 'PTS/MIN']]

df = df.sort_values(by='PTS/MIN', ascending=False)
df.head()

,PLAYER,TEAM,PTS/MIN
0,Joel Embiid,PHI,0.943452
3,Stephen Curry,GSW,0.934579
8,Giannis Antetokounmpo,MIL,0.883648
2,Donovan Mitchell,CLE,0.855153
14,Kyle Kuzma,WAS,0.853242


# Data Cleaning #

* remove retired players from the data
* remove very low play time from the data

In [309]:
# map for changing team names

team_mapping = {
    'Atlanta Hawks': 'ATL',
    'Boston Celtics': 'BOS',
    'Brooklyn Nets': 'BKN',
    'Charlotte Hornets': 'CHA',
    'Chicago Bulls': 'CHI',
    'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL',
    'Denver Nuggets': 'DEN',
    'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW',
    'Houston Rockets': 'HOU',
    'Indiana Pacers': 'IND',
    'Los Angeles Clippers': 'LAC',
    'Los Angeles Lakers': 'LAL',
    'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA',
    'Milwaukee Bucks': 'MIL',
    'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP',
    'New York Knicks': 'NYK',
    'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL',
    'Philadelphia 76ers': 'PHI',
    'Phoenix Suns': 'PHX',
    'Portland Trail Blazers': 'POR',
    'Sacramento Kings': 'SAC',
    'San Antonio Spurs': 'SAS',
    'Toronto Raptors': 'TOR',
    'Utah Jazz': 'UTA',
    'Washington Wizards': 'WAS'
}

In [317]:
# Points Data

# Minutes played is baked into the points scored, we can assume this will carry over into the next season
# Over time this will change which is why we want to always chose the directly previous season

df_pts = pts_prevtable
df_pts = df_pts[df_pts['Min'] >= 200]
df_pts

,Unnamed: 0,Player,Team,Age,GP,W,L,Min,PTS,FGM,...,REB RANK,AST RANK,TOV RANK,STL RANK,BLK RANK,PF RANK,FP RANK,DD2 RANK,TD3 RANK,+/- RANK
8,9,Stephen Curry,GSW,35,8,6,2,254.2,0.97,0.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,11,Donovan Mitchell,CLE,27,6,3,3,214.1,0.91,0.33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,16,Luka Doncic,DAL,24,7,6,1,260.2,0.85,0.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,17,Cam Thomas,BKN,22,7,3,4,239.5,0.84,0.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,18,Jayson Tatum,BOS,25,6,5,1,218.1,0.84,0.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,303,Kentavious Caldwell-Pope,DEN,30,8,7,1,260.3,0.32,0.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,306,Killian Hayes,DET,22,8,2,6,243.0,0.31,0.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
345,346,Royce O'Neale,BKN,30,7,3,4,221.2,0.26,0.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,370,Mitchell Robinson,NYK,25,7,3,4,207.4,0.21,0.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [318]:
df_pts = df_pts[['Player', 'Team', 'PTS', 'Min']]
df_pts

,Player,Team,PTS,Min
8,Stephen Curry,GSW,0.97,254.2
10,Donovan Mitchell,CLE,0.91,214.1
15,Luka Doncic,DAL,0.85,260.2
16,Cam Thomas,BKN,0.84,239.5
17,Jayson Tatum,BOS,0.84,218.1
...,...,...,...,...
302,Kentavious Caldwell-Pope,DEN,0.32,260.3
305,Killian Hayes,DET,0.31,243.0
345,Royce O'Neale,BKN,0.26,221.2
369,Mitchell Robinson,NYK,0.21,207.4


In [223]:
# PPG Data (for calculating percent of team points made)
df_ppg = ppg_table[2]

In [224]:
df_ppg = df_ppg[['Team', 'PTS']]
df_ppg['Team'] = df_ppg['Team'].map(team_mapping)
df_ppg.head()

/var/folders/2l/tqrw4qgn61q4jrsxc447rydm0000gn/T/ipykernel_69609/1477638789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ppg['Team'] = df_ppg['Team'].map(team_mapping)


,Team,PTS
0,IND,124.9
1,BOS,123.5
2,ATL,122.0
3,PHI,121.5
4,DAL,120.7


In [225]:
# Team Defense Data
df_def = def_table[2]

In [226]:
df_def = df_def[['TEAM', b'DEF\xc2\xa0RTG'.decode('utf-8')]]
df_def['TEAM'] = df_def['TEAM'].map(team_mapping)        
df_def = df_def.rename(columns={'TEAM': 'Team'})
df_def.head()

/var/folders/2l/tqrw4qgn61q4jrsxc447rydm0000gn/T/ipykernel_69609/3563296560.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_def['TEAM'] = df_def['TEAM'].map(team_mapping)


,Team,DEF RTG
0,DEN,107.5
1,DAL,113.0
2,GSW,110.2
3,BOS,105.8
4,PHI,107.3


In [207]:
# Factors affecting player point score

# Play time - last three games play time average
# Team defense - create regression describing effect of defense on points scored. Normalize for % of points scored per player
# Days of rest between games - create regression describing effect of defense on points scored. Normalize for player posessions per game *NEEDS WORK*
# Home team advantage - +2.93  =  +3.24 - 0.31 (lesser rest for travel team, accounted for by above factor) https://www.degruyter.com/document/doi/10.2202/1559-0410.1106/html




# Processing # 